<a href="https://colab.research.google.com/github/yelagampragathi/NLP_B_16/blob/main/NLP_ASS_10_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Prepare the Multilingual Text Dataset**



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
import numpy as np

# Multilingual dataset provided
data = {
    'English': [
        "What is your name?", "Where do you live?", "How old are you?",
        "What do you do for a living?", "What is your favorite color?"
    ],
    'Telugu': [
        "నీ పేరు ఏమిటి?", "నువ్వు ఎక్కడ నివసిస్తున్నావు?", "మీరు ఎంత వయస్సు ఉన్నారు?",
        "మీరు జీవించడానికి ఏమి చేస్తున్నారు?", "మీ ఇష్ట రంగు ఏది?"
    ],
    'Hindi': [
        "आपका नाम क्या है?", "आप कहाँ रहते हैं?", "आपकी उम्र क्या है?",
        "आप क्या करते हैं?", "आपका पसंदीदा रंग क्या है?"
    ],
    'Tamil': [
        "உங்கள் பெயர் என்ன?", "நீங்கள் எங்கு வாழ்கிறீர்கள்?", "உங்கள் வயது என்ன?",
        "நீங்கள் என்ன வேலை செய்கிறீர்கள்?", "உங்கள் விருப்பமான நிறம் என்ன?"
    ]
}

# Combine all sentences
all_sentences = []
for lang in data:
    all_sentences.extend(data[lang])

# Tokenize and preprocess
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(all_sentences)
sequences = tokenizer.texts_to_sequences(all_sentences)
word_index = tokenizer.word_index

# Pad sequences to make them uniform in length
max_sequence_length = max(len(seq) for seq in sequences)
padded_data = pad_sequences(sequences, maxlen=max_sequence_length)
vocab_size = len(word_index) + 1

# Parameters
embedding_dim = 64
latent_dim = 16


**Define the VAE Model**

In [ ]:
# Encoder Model
encoder_inputs = Input(shape=(max_sequence_length,))
x = Embedding(vocab_size, embedding_dim, input_length=max_sequence_length)(encoder_inputs)
x = Bidirectional(LSTM(64, return_sequences=False))(x)
z_mean = Dense(latent_dim)(x)
z_log_sigma = Dense(latent_dim)(x)

# Reparameterization trick
def sampling(args):
    z_mean, z_log_sigma = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])

# Define the encoder model
encoder = Model(encoder_inputs, [z_mean, z_log_sigma, z], name='encoder')
encoder.summary()


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_6             │ (None, 7)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 7, 64)          │          3,776 │ input_layer_6[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 128)            │         66,048 │ embedding_2[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 16)             │          2,064 │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 16)             │          2,064 │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_2 (Lambda)         │ (None, 16)             │              0 │ dense_14[0][0],        │
│                           │                        │                │ dense_15[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 73,952 (288.88 KB)

 Trainable params: 73,952 (288.88 KB)

 Non-trainable params: 0 (0.00 B)

VAE Decoder

In [ ]:
from tensorflow.keras.layers import Reshape, RepeatVector

# Decoder Model
decoder_inputs = Input(shape=(latent_dim,))
x = Dense(64, activation='relu')(decoder_inputs)

# Ensure that the Dense layer output matches max_sequence_length * embedding_dim
x = Dense(max_sequence_length * embedding_dim, activation='relu')(x)

# Reshape the output to be compatible with LSTM input
x = Reshape((max_sequence_length, embedding_dim))(x)
x = LSTM(64, return_sequences=True)(x)
x = Dense(vocab_size, activation='softmax')(x)

decoder = Model(decoder_inputs, x, name='decoder')
decoder.summary()



Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 64)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 448)                 │          29,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 7, 64)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 7, 64)               │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 7, 59)               │           3,835 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 67,067 (261.98 KB)

 Trainable params: 67,067 (261.98 KB)

 Non-trainable params: 0 (0.00 B)

**VAE Loss Function**

In [ ]:
# VAE Loss Function (KL Divergence + Reconstruction Loss)
def vae_loss(y_true, y_pred):
    # Reconstruction loss
    recon_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    recon_loss = K.mean(recon_loss)

    # KL Divergence
    kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
    kl_loss = K.mean(kl_loss) * -0.5

    return recon_loss + kl_loss


**Compile and Train the VAE**

In [ ]:
vae.compile(optimizer='adam', loss=vae_loss)

In [ ]:
# Check the shape of padded_data
print("Shape of padded_data:", padded_data.shape)

# Train the model
try:
    vae.fit(padded_data, padded_data, epochs=20, batch_size=8)
except Exception as e:
    print("Error during training:", e)

Shape of padded_data: (20, 7)
Epoch 1/20
Error during training: Tried to convert 'x' to a tensor and failed. Error: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```



**Step 3: Generating New Text with the Trained VAE**

In [ ]:
# Generate new text
def generate_text(num_samples=5):
    for _ in range(num_samples):
        z_sample = np.random.normal(size=(1, latent_dim))
        generated_sequence = decoder.predict(z_sample)
        generated_indices = np.argmax(generated_sequence[0], axis=-1)
        generated_text = tokenizer.sequences_to_texts([generated_indices])
        print("Generated Text:", generated_text[0])

# Generate new samples
generate_text(5)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
Generated Text: are జీవించడానికి జీవించడానికి జీవించడానికి क्या பெயர் a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Generated Text: where జీవించడానికి జీవించడానికి a क्या ఎంత a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Generated Text: are living a a a பெயர் a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Generated Text: favorite <OOV> a क्या क्या a a
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Generated Text: करते జీవించడానికి are a are a a
